<a href="https://colab.research.google.com/github/FairuzAthallah88/Machine-Learning-Models/blob/main/Model_ML_Sistem_Rekomendasi_Xplorin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import module

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

2. Dataset

> Gunakan dataset ini : Dataset_SistemRekomendasi_KulinerPalembang.csv
https://drive.google.com/file/d/18sTfiLHiz4DWLesZ6jSxOUTDt69wh2LI/view?usp=sharing


In [ ]:
path_dataset = '/content/Dataset_SistemRekomendasi_KulinerPalembang.csv'

try:
    df = pd.read_csv(path_dataset, delimiter=';')
    print("Dataset berhasil dimuat!")
    print(f"Bentuk data (baris, kolom): {df.shape}")

    # 5 baris pertama dari dataset
    print("\n5 Baris Pertama Data:")
    print(df.head())

    print("\nKolom-kolom yang Tersedia:")
    print(df.columns.tolist())

except FileNotFoundError:
    print(f"Error: File '{path_dataset}' tidak ditemukan. Mohon periksa kembali path file Anda.")

Dataset berhasil dimuat!
Bentuk data (baris, kolom): (209, 9)

5 Baris Pertama Data:
    Nama kuliner palembang                        Rasa        Kategori  \
0  Pempek Adaan Ayam-Udang    gurih dan ada rasa udang  makanan ringan   
1       Pempek Adaan Pedas                gurih, pedas  makanan ringan   
2       Pempek Adaan Hitam  gurih dan aroma kulit ikan  makanan ringan   
3        Pempek Adaan Tunu    gurih dan aroma panggang  makanan ringan   
4       Pempek Adaan Telur     ada tambahan rasa telur  makanan ringan   

                                           Deskripsi  \
0  Adonan adaan yang bukan 100 % ikan, melainkan ...   
1  Modifikasi adaan yang menyertakan cabai dalam ...   
2  Modifikasi adaan yang menambahkan bagian kulit...   
3  Varian "adaan" yang tidak digoreng melainkan d...   
4                       Adaan yang diisi telur kecil   

                                             Sejarah  \
0  adonan adaan yang bukan 100 % ikan, melainkan ...   
1  adaptasi untuk kon

3. Data Preprocessing

In [ ]:
kolom_kuliner = 'Nama kuliner palembang'

# Kolom-kolom yang akan digabungkan menjadi fitur
kolom_rasa = 'Rasa'
kolom_kategori = 'Kategori'
kolom_deskripsi = 'Deskripsi'
kolom_fitur = 'Kombinasi Fitur'

# Mengisi Nilai yang Hilang (NaN) menjadi String
print("Mengatasi nilai hilang (NaN)...")
df[kolom_rasa] = df[kolom_rasa].fillna('')
df[kolom_kategori] = df[kolom_kategori].fillna('')
df[kolom_deskripsi] = df[kolom_deskripsi].fillna('')

# Menggabungkan Kolom Fitur (Feature Engineering)
df[kolom_fitur] = (df[kolom_rasa] + ' ' + df[kolom_kategori] + ' ' + df[kolom_deskripsi])

# Verifikasi Akhir
if kolom_kuliner in df.columns and kolom_fitur in df.columns:
    print(f"  Preprocessing Berhasil!")
    print(f"  Kolom Kuliner (Item ID) diatur ke: '{kolom_kuliner}'")
    print(f"  Kolom Fitur (Teks Konten) diatur ke: '{kolom_fitur}'")
    print("\nGabungan Fitur :")

    # Hasil gabungan untuk 5 item pertama
    print(df[[kolom_kuliner, kolom_fitur]].head())
else:
    print(f"Error: Pastikan nama kolom sudah benar di dalam dataset.")

Mengatasi nilai hilang (NaN)...
  Preprocessing Berhasil!
  Kolom Kuliner (Item ID) diatur ke: 'Nama kuliner palembang'
  Kolom Fitur (Teks Konten) diatur ke: 'Kombinasi Fitur'

Gabungan Fitur :
    Nama kuliner palembang                                    Kombinasi Fitur
0  Pempek Adaan Ayam-Udang  gurih dan ada rasa udang makanan ringan Adonan...
1       Pempek Adaan Pedas  gurih, pedas makanan ringan Modifikasi adaan y...
2       Pempek Adaan Hitam  gurih dan aroma kulit ikan makanan ringan Modi...
3        Pempek Adaan Tunu  gurih dan aroma panggang makanan ringan Varian...
4       Pempek Adaan Telur  ada tambahan rasa telur makanan ringan Adaan y...


# New Section

4. Pembuatan vektor TF-IDF

In [ ]:
stop_words_set = stopwords.words('indonesian')
stop_words_id_list = list(stop_words_set)

# TfidfVectorizer dengan Stop Words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_id_list, ngram_range=(1, 2))

# Fit dan transform di kolom fitur
tfidf_matrix = tfidf_vectorizer.fit_transform(df[kolom_fitur])

# Ukuran matriks TF-IDF
print("Ukuran matriks TF-IDF:", tfidf_matrix.shape)
print(f"Catatan: Matriks berisi {tfidf_matrix.shape[0]} kuliner dan {tfidf_matrix.shape[1]} kata/frasa unik.")

Ukuran matriks TF-IDF: (209, 1893)
Catatan: Matriks berisi 209 kuliner dan 1893 kata/frasa unik.


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


5. Perhitungan Cosine Similarity

In [ ]:
# Menghitung cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Ukuran matriks Cosine Similarity:", cosine_sim.shape)
print("\nContoh matriks Cosine Similarity (5x5 pertama):")
print(cosine_sim[:5, :5])

Ukuran matriks Cosine Similarity: (209, 209)

Contoh matriks Cosine Similarity (5x5 pertama):
[[1.         0.07295738 0.08525631 0.07933521 0.08051324]
 [0.07295738 1.         0.24315807 0.09535672 0.09677265]
 [0.08525631 0.24315807 1.         0.12577943 0.06459062]
 [0.07933521 0.09535672 0.12577943 1.         0.10523238]
 [0.08051324 0.09677265 0.06459062 0.10523238 1.        ]]


6. Fungsi Rekomendasi

In [ ]:
indices = pd.Series(df.index, index=df[kolom_kuliner]).drop_duplicates()

def dapatkan_rekomendasi(kuliner, cosine_sim_matrix=cosine_sim, jumlah_rekomendasi=10):
    """
    Fungsi untuk mendapatkan rekomendasi item berdasarkan kemiripan konten.

    Args:
        kuliner (str): Nama kuliner Palembang yang ingin dicari rekomendasinya.
        cosine_sim_matrix (numpy.ndarray): Matriks cosine similarity hasil perhitungan TF-IDF.
        jumlah_rekomendasi (int): Jumlah item yang ingin direkomendasikan.

    Returns:
        pandas.Series: Series berisi nama kuliner yang direkomendasikan.
    """
    # Mengecek apakah judul ada di dalam dataset
    if kuliner not in indices:
        return f"Item dengan kuliner '{kuliner}' tidak ditemukan."

    # Mendapatkan indeks dari item yang cocok dengan kuliner
    idx = indices[kuliner]

    # Mendapatkan skor kesamaan dari semua item dengan item tersebut
    # enumerate untuk membuat pasangan (indeks, skor_kesamaan)
    skor_kesamaan = list(enumerate(cosine_sim[idx]))

    # Mengurutkan item berdasarkan skor kesamaan (dari tertinggi ke terendah)
    skor_kesamaan = sorted(skor_kesamaan, key=lambda x: x[1], reverse=True)

    # Mengambil skor dari `jumlah_rekomendasi` item teratas
    skor_kesamaan = skor_kesamaan[1:jumlah_rekomendasi+1]

    # Mendapatkan indeks dari item-item yang direkomendasikan
    item_indices = [i[0] for i in skor_kesamaan]

    # Mengembalikan judul dari item yang paling mirip
    return df[kolom_kuliner].iloc[item_indices]

print("Fungsi rekomendasi berhasil dibuat!")


Fungsi rekomendasi berhasil dibuat!


7. Testing hasil

In [ ]:
# Pilih salah satu kuliner dari dataset untuk mendapatkan rekomendasi
nama_kuliner_pilihan = "Model"

# Jumlah rekomendasi yang ingin ditampilkan
jumlah_rekomendasi_tampil = 7

# --- Kode Rekomendasi Ringkas ---
if nama_kuliner_pilihan not in indices:
    print(f"Error: Kuliner dengan nama '{nama_kuliner_pilihan}' tidak ditemukan dalam dataset.")
else:
    # Mendapatkan rekomendasi
    rekomendasi_series = dapatkan_rekomendasi(nama_kuliner_pilihan, jumlah_rekomendasi=jumlah_rekomendasi_tampil)

    # Menampilkan informasi item yang dipilih
    print(f"======================================================================")
    print(f"Rekomendasi Berdasarkan Item Kuliner: '{nama_kuliner_pilihan}'")
    if kolom_kuliner:
        kategori_pilihan = df[kolom_kuliner].iloc[indices[nama_kuliner_pilihan]]
        print(f"Kategori Acuan: {kategori_pilihan}")
    print(f"======================================================================")

    # Menampilkan header tabel hasil
    print(f"{'No.':<4} {'Kuliner Rekomendasi':<50} {'Skor Kemiripan':<25}")
    print(f"-"*90)

    # Iterasi melalui hasil rekomendasi untuk menampilkan detailnya
    for i, kuliner_rekomendasi in enumerate(rekomendasi_series):
        # Dapatkan indeks dari item pilihan dan item rekomendasi
        idx_pilihan = indices[nama_kuliner_pilihan]
        idx_rekomendasi = indices[kuliner_rekomendasi]

        # Ambil skor kemiripan
        skor_kemiripan = cosine_sim[idx_pilihan][idx_rekomendasi]

        # Tampilkan hasil dalam format tabel
        print(f"{i+1:<4} {kuliner_rekomendasi:<50} {skor_kemiripan:<20.4f}")

        # Tampilkan kategori jika kolomnya ada
        if kolom_kuliner:
            kuliner_rekomendasi = df[kolom_kuliner].iloc[idx_rekomendasi]
            print(f"{'':<4} └ Kuliner: {kuliner_rekomendasi}")

Rekomendasi Berdasarkan Item Kuliner: 'Model'
Kategori Acuan: Model
No.  Kuliner Rekomendasi                                Skor Kemiripan           
------------------------------------------------------------------------------------------
1    Celimpungan Kuah Bening Campur Santan              0.1860              
     └ Kuliner: Celimpungan Kuah Bening Campur Santan
2    Tekwan Kuah Kaldu Udang                            0.1722              
     └ Kuliner: Tekwan Kuah Kaldu Udang
3    Ikan Salai Kuah Asam                               0.1576              
     └ Kuliner: Ikan Salai Kuah Asam
4    Pempek Pistel                                      0.1504              
     └ Kuliner: Pempek Pistel
5    Martabak HAR Tanpa Kuah (Kering)                   0.1414              
     └ Kuliner: Martabak HAR Tanpa Kuah (Kering)
6    Tekwan Kuah Bengkuang                              0.1316              
     └ Kuliner: Tekwan Kuah Bengkuang
7    Pindang Patin                               

In [ ]:
import joblib
import os

# Define paths for saving models
output_dir = 'recommendation_models'

def save_recommendation_models(vectorizer, tfidf_mat, cos_sim, dataframe, index_series, output_directory=output_dir):
    """
    Saves all recommendation system components to disk.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    joblib.dump(vectorizer, os.path.join(output_directory, "tfidf_vectorizer.pkl"))
    joblib.dump(tfidf_mat, os.path.join(output_directory, "tfidf_matrix.pkl"))
    joblib.dump(cos_sim, os.path.join(output_directory, "cosine_sim.pkl"))
    joblib.dump(dataframe, os.path.join(output_directory, "data.pkl"))
    joblib.dump(index_series, os.path.join(output_directory, "indices.pkl"))
    print(f"All models and data successfully saved to '{output_directory}'!")

def load_recommendation_models(output_directory=output_dir):
    """
    Loads all recommendation system components from disk.
    """
    try:
        loaded_tfidf_vectorizer = joblib.load(os.path.join(output_directory, "tfidf_vectorizer.pkl"))
        loaded_tfidf_matrix = joblib.load(os.path.join(output_directory, "tfidf_matrix.pkl"))
        loaded_cosine_sim = joblib.load(os.path.join(output_directory, "cosine_sim.pkl"))
        loaded_df = joblib.load(os.path.join(output_directory, "data.pkl"))
        loaded_indices = joblib.load(os.path.join(output_directory, "indices.pkl"))
        print(f"All models and data successfully loaded from '{output_directory}'!")
        return loaded_tfidf_vectorizer, loaded_tfidf_matrix, loaded_cosine_sim, loaded_df, loaded_indices
    except FileNotFoundError:
        print(f"Error: One or more model files not found in '{output_directory}'.")
        return None, None, None, None, None

def test_loaded_models(kuliner_pilihan="Model", jumlah_rekomendasi=7):
    """
    Tests the recommendation function with the loaded models and data.
    """
    print("\n--- Testing Loaded Models ---")
    loaded_tfidf_vectorizer, loaded_tfidf_matrix, loaded_cosine_sim, loaded_df, loaded_indices = load_recommendation_models()

    if loaded_cosine_sim is None or loaded_indices is None or loaded_df is None:
        print("Failed to load models. Cannot perform test.")
        return

    # Re-define the recommendation function to use loaded components
    def get_recommendations_from_loaded(kuliner, cosine_sim_matrix, index_series, dataframe, num_recommendations=10):
        if kuliner not in index_series:
            return f"Item with kuliner '{kuliner}' not found in loaded data."
        idx = index_series[kuliner]
        skor_kesamaan = list(enumerate(cosine_sim_matrix[idx]))
        skor_kesamaan = sorted(skor_kesamaan, key=lambda x: x[1], reverse=True)
        skor_kesamaan = skor_kesamaan[1:num_recommendations+1]
        item_indices = [i[0] for i in skor_kesamaan]
        return dataframe['Nama kuliner palembang'].iloc[item_indices]

    if kuliner_pilihan not in loaded_indices:
        print(f"Error: Kuliner dengan nama '{kuliner_pilihan}' tidak ditemukan dalam dataset yang dimuat.")
    else:
        rekomendasi_series_loaded = get_recommendations_from_loaded(
            kuliner_pilihan, loaded_cosine_sim, loaded_indices, loaded_df, num_recommendations=jumlah_rekomendasi # FIX: Changed 'num_recommendasi' to 'num_recommendations'
        )

        print(f"=====================================================================")
        print(f"Rekomendasi Berdasarkan Item Kuliner (dari model yang dimuat): '{kuliner_pilihan}'")
        kategori_pilihan_loaded = loaded_df['Nama kuliner palembang'].iloc[loaded_indices[kuliner_pilihan]]
        print(f"Kategori Acuan: {kategori_pilihan_loaded}")
        print(f"=====================================================================")

        print(f"{'No.':<4} {'Kuliner Rekomendasi':<50} {'Skor Kemiripan':<25}")
        print(f"-"*90)

        for i, kuliner_rekomendasi in enumerate(rekomendasi_series_loaded):
            idx_pilihan = loaded_indices[kuliner_pilihan]
            idx_rekomendasi = loaded_indices[kuliner_rekomendasi]
            skor_kemiripan = loaded_cosine_sim[idx_pilihan][idx_rekomendasi]
            print(f"{i+1:<4} {kuliner_rekomendasi:<50} {skor_kemiripan:<20.4f}")
            print(f"{'' :<4} └ Kuliner: {kuliner_rekomendasi}")

# --- Execute save and test ---
# First, save the current models and data
save_recommendation_models(tfidf_vectorizer, tfidf_matrix, cosine_sim, df, indices)

# Then, test the loading and recommendation process
test_loaded_models()

All models and data successfully saved to 'recommendation_models'!

--- Testing Loaded Models ---
All models and data successfully loaded from 'recommendation_models'!
Rekomendasi Berdasarkan Item Kuliner (dari model yang dimuat): 'Model'
Kategori Acuan: Model
No.  Kuliner Rekomendasi                                Skor Kemiripan           
------------------------------------------------------------------------------------------
1    Celimpungan Kuah Bening Campur Santan              0.1860              
     └ Kuliner: Celimpungan Kuah Bening Campur Santan
2    Tekwan Kuah Kaldu Udang                            0.1722              
     └ Kuliner: Tekwan Kuah Kaldu Udang
3    Ikan Salai Kuah Asam                               0.1576              
     └ Kuliner: Ikan Salai Kuah Asam
4    Pempek Pistel                                      0.1504              
     └ Kuliner: Pempek Pistel
5    Martabak HAR Tanpa Kuah (Kering)                   0.1414              
     └ Kuliner: Mart

In [ ]:
# import joblib

# joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")
# joblib.dump(tfidf_matrix, "tfidf_matrix.pkl")
# joblib.dump(cosine_sim, "cosine_sim.pkl")
# joblib.dump(df, "data.pkl")
# joblib.dump(indices, "indices.pkl")

# print("Semua model berhasil disimpan!")


Semua model berhasil disimpan!


In [ ]:
# from google.colab import files

# files.download("tfidf_vectorizer.pkl")
# files.download("tfidf_matrix.pkl")
# files.download("cosine_sim.pkl")
# files.download("data.pkl")
# files.download("indices.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>